In [1]:
from proc_v1 import p, target, X_cat, X_cat2, X_num, X_all, sc
from ml_v2 import config, xgb_adapter, lgb_adapter, cb_adapter, nn_adapter, lr_adapter, scheduler, mapk_prob, ss, skf, train_data_proc
from sklearn.preprocessing import LabelEncoder
from functools import partial
import sgml
import numpy as np
import pandas as pd
le = LabelEncoder()
target_l = target + '_l'
df_train = p.fit_transform(['data/train.csv'])
df_org = p.transform(['data/Fertilizer Prediction.csv'])
df_test = p.transform(['data/test.csv'])
df_train = df_train.assign(
    **{target_l: lambda x: le.fit_transform(x[target])}
)
df_org = df_org.assign(
    **{target_l: lambda x: le.transform(x[target])}
)

config['train_data_proc'] = partial(train_data_proc, df_org = df_org)

2025-06-08 20:35:43.614884: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-08 20:35:43.624845: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749382543.636607  223665 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749382543.639954  223665 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1749382543.649291  223665 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

## XGB4_ORG_TGT_CAT2

In [8]:
result = sgml.cv(
    df_train, ss, {
        'model_params': {
            'n_estimators': 10000, 'max_depth': 7, 'enable_categorical': True, 'learning_rate': 0.03,
            'early_stopping_rounds': 100, 'objective': 'multi:softprob', 'colsample_bytree': 0.4, 'random_state': 13, 'tree_method': 'hist'
        }, 'X_num': X_num, 'X_cat': X_cat, 'X_tgt': X_cat2, 'tgt': {'target_type': 'multiclass'},
        'train_data_proc_param': {'org': True},
        'validation_fraction': 'oof',
    }, config, xgb_adapter, use_gpu = True, rerun = False,
    desc = 'XGB4_ORG + TGT_CAT2'
)
result

{'valid_scores': [0.3516866666666666],
 'valid_prd':                0         1         2         3         4         5         6
 id                                                                          
 0       0.115019  0.188122  0.104235  0.145152  0.209115  0.105596  0.132761
 3       0.134706  0.181463  0.159142  0.148759  0.139315  0.095768  0.140848
 9       0.158562  0.123670  0.122395  0.217930  0.184850  0.094924  0.097668
 22      0.184135  0.131195  0.142453  0.184219  0.183537  0.069698  0.104763
 30      0.227304  0.168374  0.102963  0.134856  0.121174  0.110359  0.134970
 ...          ...       ...       ...       ...       ...       ...       ...
 749939  0.172990  0.181815  0.200594  0.105863  0.142309  0.109585  0.086844
 749981  0.193619  0.112732  0.189392  0.111378  0.148112  0.130461  0.114306
 749990  0.161220  0.142322  0.173247  0.155457  0.171035  0.137301  0.059418
 749991  0.124259  0.174866  0.158693  0.199910  0.118758  0.120081  0.103433
 749999  0.2

- 성능 차이가 너무 커서 쓰지 않겠습니다.